# Load and Prepare the Data

In [6]:
# pip install datasets  transformers

In [10]:
import pandas as pd
from datasets import Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling

# Load data and prepare the combined text column
df = pd.read_csv('reviews_supplements.csv')
df = df.dropna(subset=['text', 'title', 'rating'])  # Drop rows where any of these columns are NaN
df['combined_text'] = df['title'].astype(str) + " [Title] " + df['text'].astype(str) + " [Rating: " + df['rating'].astype(str) + "]"

# Initialize the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# If your tokenizer does not have a pad token, the following line sets it to use the EOS token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Tokenization function that handles each example
def tokenize_function(examples):
    tokens = tokenizer(examples['combined_text'], truncation=True, padding="max_length", max_length=512)
    # Set the labels equal to input_ids because we are doing causal language modeling
    tokens['labels'] = tokens['input_ids'].copy()  # Labels are shifted input_ids
    return tokens

# Convert DataFrame to Dataset and tokenize
dataset = Dataset.from_pandas(df)
dataset = dataset.map(tokenize_function, batched=True, batch_size=16)

# Remove columns that are not used by the model
dataset = dataset.remove_columns(['user_id', 'title', 'timestamp', 'combined_text', 'time', 'date', 'rating', 'text', 'asin', 'verified_purchase', 'parent_asin', 'helpful_vote', '__index_level_0__'])

# Split the dataset
train_test_split = dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

# Prepare a data collator for dynamic padding
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="epoch",
)

# Initialize the model
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Train the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained('./fine_tuned_gpt2')
tokenizer.save_pretrained('./fine_tuned_gpt2')


C:\Users\kumar\anaconda3\lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading file vocab.json from cache at C:\Users\kumar/.cache\huggingface\hub\models--gpt2\snapshots\607a30d783dfa663caf39e06633721c8d4cfcd7e\vocab.json
loading file merges.txt from cache at C:\Users\kumar/.cache\huggingface\hub\models--gpt2\snapshots\607a30d783dfa663caf39e06633721c8d4cfcd7e\merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at C:\Users\kumar/.cache\huggingface\hub\models--gpt2\snapshots\607a30d783dfa663caf39e06633721c8d4cfcd7e\tokenizer_config.json
loading configuration file config.json from cache at C:\Users\kumar/.cache\huggingface\hub\models--gpt2\snapshots\607a30d783dfa663ca

Map:   0%|          | 0/16660 [00:00<?, ? examples/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file config.json from cache at C:\Users\kumar/.cache\huggingface\hub\models--gpt2\snapshots\607a30d783dfa663caf39e06633721c8d4cfcd7e\config.json
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summar

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 